# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [4]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    
    
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print('到達頁面底端')
        break
    else:
        print('網頁更新中...')
        last_height = new_height
        continue
    

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [5]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [6]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 對抗低利率時代！美元下半年走弱　投資人還可布局這些外幣

▲在後疫情時代，投資人關注外幣該如何配置才能降低風險。（示意圖／記者張一中攝）
記者紀佳妘／台北報導
今年初受到疫情影響，全球金融市場動盪不安，不過如今各國逐步重啟經濟，推出各項救市措施，多家外銀預估經濟將呈現「勾勾型」或「U型」復甦，但下半年仍有中美關係緊張、美國大選不確定性等變數，投資人關注外幣該如何配置才能降低風險。
瑞銀財富管理投資總監辦公室外匯分析師陳得能指出，美國聯準會主席鮑威爾早已表明「想都不敢去想加息」，央行預期通脹在2022年前明顯低於目標2%，失業率也保持在偏高水準，美國在未來2年半根本難以加息，相信利率在更長的時間內保持不變。
陳得能表示，儘管部分地區疫情再度升溫，使得避險資金加快流向美元，但美國聯準會釋出保持流動性充裕的強烈意願，不容許3月美元危機再次發生，因此美元進一步上漲空間有限，避險貨幣選擇繼續以日元及瑞士法郎為首選。
滙豐私人銀行亞洲區首席市場策略師范卓雲也認為，受惠於史無前例的超長寬鬆政策，預期明年經濟將穩步復甦，因此美元大幅修正的風險有限，將投資觀點從「轉強」轉為「中性」。
渣打銀行財富管理處負責人陳太齡也預期，在美國經濟基本面優勢不再、美元利差優勢消退、各國央行同步採行空前的寬鬆貨幣政策救市等3大因素影響，削減美元避險的需求，預估未來12個月美元指數可能下跌5至7%。
不過，陳太齡相對看好歐元、英鎊、澳幣後續走勢，隨著政治風險降低，讓歐元作為美元替代品更具吸引力，預估下半年歐元兌美元可望反彈至1.19；被嚴重低估的英鎊，隨著美元走貶，可能受到外國資本流入支撐，英鎊兌美元則有機會挑戰1.32；澳幣已扭轉長期跌勢緩步回升，預估未來12個月有機會升至0.74左右；至於人民幣短期內則可能落在6.9至7.2區間格局。
----------------------------------------------------------------------

[1] 大同二大市場派搶開股東臨時會　經濟部處理原則曝光

▲大同公司爆發經營權之爭，二大市場派已向經濟部遞件申請召開臨時股東會。（圖／記者湯興漢攝）
記者林淑慧／台北報導
大

▲佐登-KY捐贈家總5000份長照精油舒壓包。（圖／佐登提供）
記者姚惠茹／台北報導
佐登-KY（4190）今（17）日捐贈中華民國家庭照顧者關懷總會(家總)長照精油紓壓包5000份，並發動全台631位美容師化身宣傳志工，協助照顧者紓壓教學喘息服務，以實際行動幫助主要照顧者。談到營運，執行長陳佳琦表示，預計年底前增設7家醫美診所，並看好下半年營運可望優於上半年。
佐登-KY執行長陳佳琦表示，今年上半年中國大陸受到新冠肺炎疫情衝擊，所幸台灣市場完全不受影響，所以今年下半年計畫加速台灣醫美診所的展店速度，預計年底前將在台北、桃園、新竹、台中、員林、台南、高雄開設醫美診所，並看好下半年營運可望優於上半年。
▲佐登-KY執行長陳佳琦（中）。（圖／記者姚惠茹攝）
佐登-KY今年與家總合作，全面啟動長照關懷計畫， 包含全通路長照議題宣傳計畫，發揮145家門市與電商通路優勢，化身社區長照小站，並將培訓631位門市美容師成為長照宣導種子講師，同時投入志工喘息服務計畫，捐贈家總5,000份長照精油紓壓包，並推動員工一年兩天志工假做公益。
陳佳琦表示，佐登深耕台灣32年，為了善盡企業社會責任，持續關懷弱勢團體，並從長期照顧認知與需求調查發現，女性在長照課題中負擔非常多的責任，也更需要喘息服務，若公益能結合企業本身的優勢，效果一定事半功倍。
家總秘書長陳景寧表示，受到華人社會傳統觀念影響，容易將長期照顧視為女性責任，因此家總呼籲民眾，其實人人都可以成為主要照顧者，並多使用「家庭照顧協議」，共同商討醫療決定、照顧安排、家人分工、花費分擔等問題，減輕主要照顧者的壓力。
►過期票券別浪費！簡單3步驟　教你換1.5倍東森幣
----------------------------------------------------------------------

[11] 股神巴菲特投資攻略曝！「財報3大重點」衡量企業價值　再看6指標聰明選股

▲「股神」巴菲特。（圖／路透）
記者曾筠淇／綜合報導
▲巴菲特特別重視財報上的股東權益報酬率、營業利益率及業主盈餘。（示意圖／取自Pixabay）
第二為「營業利益率」，其重點為「只考量本業收支狀況而不考慮公司副業」，通常是檢視公司是否將股東的錢確實運用在本業並創造獲利，而非投資非本業而獲利，公式為「營業利益/營收」，計算出的數值越大，則代表本業獲利能

文／莊孟翰
最近有位購屋者以六百萬元購買新竹一棟中古透天厝，入住後赫然發現屋旁不遠處密林中有三座墳墓，在身心備受煎熬，加以房價嚴重貶損情況下，遂向新竹地院提告，要求減少價金25％，156萬元。
訴訟過程買方表示簽約前曾數度詢問賣方及房仲業周邊有無嫌惡設施，答案都是「無」，賣方應訊時則表示鄰近山坡是國有地，上面既有的墳墓是眾所皆知的事，不致影響房價。法官在審理過程為求慎重起見，特委託不動產估價師事務所現勘，認定三處墳墓與房屋距離為二十至三十公尺，再就可視性、遮蔽性與管理性進行鑑價，認為影響房價比例約為8.5％，法官遂判決賣方應賠償42萬6千元。
另一案例也是相當值得參考，其實「NIMBY」(Not In My Back Yard)「不要在我家後院」一向都是房地產業與購屋者所關切的焦點所在。
知名藝人周杰倫的媽媽，2008年在新北市淡水區購買「台北灣」一戶21樓96坪、2756萬元的預售景觀豪宅，三年後交屋時打開窗戶，竟然發現屋外是一片亂葬崗，觀景頓時變成「觀墳」，周媽媽認為建商隱瞞實情，要求解約退款，建商卻反要求支付15％違約金，其因在於交屋時這片墳墓已經遷移，所以反過來要求買方支付違約金。法院開庭時訊問有無去過現場勘察地形，律師回答簽約前有到工地附近查看，但亂葬崗被前排樹木與大型看板遮擋，根本看不出背後另有玄機，因此，主張建商明顯隱匿事實。至於最後結果如何，就有待司法判決。
其實在當前整體房市超額供給依舊相當嚴重情況下，除非有特殊需求，否則一向被列為嫌惡設施的墳墓，即使減價一、二成，必然還是乏人問津，其因在於兼具景觀價值的「心理價格」，尤其是如果有長者同住，心理影響因素必然愈大！
姑且不論這些事件如何落幕，購屋之前應切記以下三大重要事項：
1.應牢記貨比十家的寶貴經驗：最近有位會計師朋友告訴我買房之前看過一百戶以上，有時一天連看五、六戶，愈看愈有心得，最後連區位、格局、價格、家電贈品等，該爭取的一樣都沒少，令他相當滿意。
2.審閱期五天好好做功課：尤其是預售屋，更應請教專家幫忙審閱，以免交屋時與預售樣品屋落差太大，而有被騙的感覺。
3.詳細閱讀「不動產標的現況說明書」，尤其房屋所在位置如緊鄰山坡地，更應注意是否為順向坡，以免天候異常發生土石流，導致房產大幅折損。
就一般購屋者而言，對於鄰避設施(NIMBY)一向都是避之唯恐不及，幾乎沒有任何妥協餘地，尤其如有長者同住

▲台灣微軟總經理孫基康（左）、台灣微軟人工智慧研發中心執行長張仁炯（右）。（圖／記者邱倢芯攝）
記者邱倢芯／台北報導
還記得2019年9月台灣微軟於在台成立30週年記者會時，喊出會以高於業界的價碼廣徵AI人才嗎？在經過了一陣疫情影響後，勢必有許多人好奇，台灣微軟是否將縮減對於台灣人才投資的力道？對此，台灣微軟人工智慧研發中心執行長張仁炯就笑著表示，微軟向來投資就是哪裡有最好人才、哪裡有虹吸效應、哪裡有人才群聚效應我們就有投資，「所以從來都沒有錢的問題」。
張仁炯指出，微軟一直都有持續在台灣投資，即便是過去這幾個月因為疫情狀況，整個國際間多數公司都停止招聘，微軟內部談論的仍是「如何在台灣增加人力？」
不過疫情勢必對於公司營收會有所影響，為什麼微軟仍願意在台灣擴大徵才？張仁炯解釋，其實微軟股票一直都有在漲，市值不是第一就是第二，所以徵才從來就沒有錢的問題。至於為什麼選擇台灣，就是因為台灣的人才相當充足，微軟在台灣看到了許多投資的誘因。
據了解，台灣微軟的AI團隊目前已有超過100人，且團隊當中有95%是台灣人才，僅有5%是較資深的國外研究員。外國人才比例較少，是因為微軟都已在台灣設立研究團隊了，就是看上台灣有很多AI人才；張仁炯認為，如果微軟對台灣人才沒信心，人才要從國外拉來台灣，這樣就沒有意義了。
不過即便AI團隊已有超過百人的規模，但是台灣微軟仍不停止在台灣徵才；張仁炯強調，台灣微軟廣徵AI人才，不會從「人數」的角度來看，而是會從公司、產業的角度來思考，只要有投資的意義，擴大徵才的投資就會持續。
張仁炯表示，台灣有非常好的製造業，這些企業如果願意做智慧轉型，將科技導入製造業場景，無論是製造過程節省人力、然後提高產品品質，那最終就可以提高收益，並進一步地提升競爭力。
另外，張仁炯也注意到這波肺炎疫情對於台灣企業的影響，以及台灣中小企業對於數位轉型的作法；他認為，其實台灣企業一直都有許多轉型的機會，只是過去大家沒有意識到或是做得比較慢。
不過在疫情過後，有部分的企業似乎開始意識到數位轉型的重要性；有些企業主會認為，因為疫情的關係，現在公司的營運慢了下來，公司該如何「跑快一點」？
張仁炯認為，在疫情的影響後，企業該去思考如何做對的、更小心的投資，且有遠見的企業會認真投資，不像以前僅投資七八成；微軟也會視企業需求，把重要的數位轉型應用帶來台灣，以利台灣產業的需求。
▼微軟除

▲中華電2020「IoT大平台創意應用大賽」徵件開跑。（圖／翻攝自中華電信官網）
記者邱倢芯／台北報導
台灣已進入5G世代，中華電信為推動物聯網教育，積極投入5G、AI等應用研發，將再舉辦年度品牌賽事「IoT大平台創意應用大賽」，並祭出155萬元高額獎金期望吸引學子、社會創作者，運用IoT大平台激發多元創意，使創作更符合5G物聯網產業應用；賽事徵件日期自即日起至9/11止，期望在5G及各領域開創嶄新應用服務。
中華電信董事長謝繼茂表示，因應5G來臨，特別於本屆賽事增設「行動物聯特別獎」，鼓勵參賽者以NB-IoT激發更多創新應用及商業模式，使創作更符合5G物聯網產業應用，滿足智慧城市全方位的需求。
此次的賽事組別分為校園組與社會組，無論是在校學生或應屆畢業生、社會人士或一般個人、新創團隊、企業組織、相關產業人士及各領域的Makers皆可參加，參賽者可就智慧城市、智慧家庭、智慧健康、智慧交通、智慧安防、智慧農業、智慧環境、智慧校園、智慧監測、智慧追蹤等面向進行研發創作。
另外，參賽者作品只要使用NB-IoT技術，即可額外享有「行動物聯特別獎」參賽資格，該獎項與原組別可同時獲獎，最高可獲得39萬元。中華電信IoT大平台創意應用大賽，總獎金高達155萬元。
----------------------------------------------------------------------

[36] Netflix Q2繳出淨利年增1.6倍佳績　盤後股價卻暴跌近1成原因曝光

▲ Ｎetflix於第2季繳出漂亮的成績單。（圖／路透）
記者林鼎閎／綜合報導
影音串流平台Netflix發佈了2020年第2季的財報。財報顯示，Netflix第2季營收為61.5億美元，比去年同期的49.2億美元年增24.9%；凈利潤為7.2億美元，與去年同期的2.71億美元繳出年增166%的成績單。儘管Netflix第2季度營收超出分析師預期，但其每股收益和第3季的營收展望都未能達到預期，導致美國時間16日盤後股價大跌近10%。
發布第2季財報的同時，Netflix也提供了2020年第3季的業績預期，預計營收為63.3億美元，低於分析師預期的64億美元；凈收入為9.54億美元，每股收益為2.09美元，高於分析師預期的2.01美元。Netflix對訂戶增加的預測和分析師的預期可以說是南轅北轍

▲民眾於超商領取紙本三倍券 。（圖／記者張一中攝）
記者林淑慧／台北報導
振興三倍券今（17）日為開放領取實體券第三天，依據經濟部統計，截至今天上午11時為止，全台申購已突破1396萬2129人次，已有超過523萬人完成紙本領券；官員提醒，收到三倍券商家23日起就能到開戶銀行、郵局、農會兌換成現金，全台有超過3000個據點服務。
依據經濟部統計，目前全台申購三倍券已達1396萬2129人次，其中透過官網預購者有53萬3557人，選擇四大超商預購者有981萬3853人；至於數位綁定共173萬0488人，含信用卡115萬8171人，電子票證23萬0905人，行動支付34萬1421人。
官員指出，三倍券上路第三天，目前累積領取紙本券的民眾已突破523萬人，其中郵局售出188萬4222份、394萬7861人選擇在超商超市取券。
針對三倍券對於振興經濟的目標，經濟部長王美花今天接受廣播專訪時表示，「效果是好的，非常好，」而政府為了發放三倍券事前做了很多準備工作，包括郵局1000多個據點提供貼心服務，提供椅子、搭設棚子遮陽，不少排隊的長輩邊聊天邊等候領券。
經濟部表示，三倍券預定從本月23日起可兌換，有統一編號店家業者，可到金融機構填寫三倍券背面資訊及兌領單，並提供相關證明文件（如稅籍登記或商業登記），金額存入負責人帳戶。無統一編號店家業者，可委託合格的產業公（工、協）會等組織，前往金融機構協助兌換，金額存入受委託組織帳戶，由受委託組織發給委託店家業者。
 
►過期票券別浪費！點這邊...幫你搶救變身
 
----------------------------------------------------------------------

[48] 3月股市觸底以來　股神巴菲特單靠蘋果就大賺400億美元

▲「股神」巴菲特（Warren Buffett）3月抄底，從蘋果大賺400億美元。（圖／路透社） 
記者林鼎閎／綜合報導
巴菲特（Warren Buffett）有史以來最大的一筆交易就是大舉收購蘋果股票。自三月份市場觸底以來，波克夏公司持有蘋果的股份（現已佔其股票投資組合的40％）的市值增加了400億美元。這項投資幫助該集團度過新冠肺炎的衝擊，發揮了至關重要的作用，因為其業務的其他投資（包括保險和能源）都受到了非常大的影響。
投資如此高調的股票違反了巴菲特著名的價值投

記者黃可昀／綜合報導
租屋族每個月除了要繳房租，還得負擔一度5元至6元不等的電費。台中一名網友表示，當初他租下這間8坪6000元套房，簽約時說好一度電5元，結果房東說他用電量太大，必須漲到一度5.5元，當時他摸摸鼻子答應調漲要求，想不到房東變本加厲，最近更是漲到一度6.5元，讓他實在忍無可忍。
原PO在臉書社團《爆廢公社二館》表示，一開始租約是載明電費一度5元，後來漲到5.5元他也照付，「今天房東更扯了！說我用電一個月使用了500多度，電費變一度6.5元！」從房東給的單據來看，他6月總共用了518度，乘上一度6.5元，光是電費一個月就高達3367元。
讓他最納悶的是，房間內只有冷氣、冰箱、電視以及電風扇4樣電器，冷氣和風扇只有回到家才會開，電視觀看時間也不長，只有冰箱是插著電24小時運轉，怎麼想都覺得用電量不可能破500度。
向房東反應後，對方不悅表示，「請台電來查看看」，還威脅如果請相關單位來調查，那麼查完就搬走別繼續住了，「有大大們能否給個看法！重點一個月500多度的電，我實在想不出來電用在哪。」他也附上家裡瞬熱式熱水器、窗型冷氣、冰箱的照片，都屬於耗電量偏大的電器類型。
網友先是對於用電量表示懷疑，「我兩個月也沒用到500度啊」、「500度有點誇張，200度就很勉強了，說好5塊就不能調，不然就不要租了」、「扣掉廁所不到8坪，一個人住一個月500多度真的有誇張。」此外，電費一度6.5元也已超過現行法規的租屋電費上限，內政部7月15日已公告新規定，其中包含「電費不得超出台電所公告之上限」，按照今年資料，夏季電價最高一度6.4元；非夏季一度5.3元。
房東第一次只漲到5.5元，即使低於6.4元也已經違法，因為當初簽約說好一度5元就是5元，日後不可用任何理由調漲，「合約打多少就是多少，要漲也是合約結束在（再）漲」、「一開始說多少就應該多少，另外電錶看一下規格對不對，這價格都可以換半台新的省電變頻了。」
----------------------------------------------------------------------

[59] 中芯登A股揭「紅色晶片國家隊」序幕　謝金河：對台半導體業衝擊正式開始

▲中芯國際登陸A股科創版，首日股價收漲202%。（圖／CFP）
中國大陸最大晶圓代工廠中芯國際昨（16）日正式登陸上海科創板，開盤股價狂飆245.

▲票券金融公司近年外幣需求大幅增加，金管會將對票金公司進行2項開放。（圖／資料照）
記者紀佳妘／台北報導
票券金融公司近年外幣需求大幅增加，金管會今（16）日表示，為強化票券金融公司辦理外幣債券業務的資金調度能力，將對票券金融公司進行2項開放，一是可與OBU及OSU辦理外幣債券買賣及其附條件交易，二是與OBU辦理外幣資金拆借。
金管會銀行局表示，票券金融公司近年因外幣債券部位大幅成長，確有外幣資金的需求，但實務上銀行及證券商的外幣債券部位、銀行的外幣資金主要列於OBU（國際金融業務分行）及OSU（國際證券業務分公司）帳上，導致票券金融公司無法透過外幣債券附買回交易向國內外匯指定銀行（DBU）及國內證券商（DSU）借入外幣資金，及與DBU承作外幣資金拆借，來滿足其外幣資金需求。
根據金管會統計，觀察近5年票金公司持有外幣債券部位餘額變化，2015年來到209億元，2016年增至652億元，到了2017年稍微縮減至628億元，而2018年又成長到704億元，2019年更是一舉突破千億元關卡，來到1,032億元，到今年第2季為止，持續成長至1,082億元。
依現行國際金融業務條例第4條第1項第6款規定，OBU得辦理與「其他金融機構」的資金借貸及外幣有價證券的買賣等業務；及第22條之4第1項第3款規定，OSU得辦理與「其他金融機構」外幣有價證券的買賣等業務。
不過，金管會2014年11月24日發布的解釋令，「其他金融機構」的範圍並未包含票券金融公司，因此票券金融公司無法於OBU及OSU取得外幣資金。
銀行局指出，考量票券業的業務需求，以及相關業務風險已有規範可資控管，因此將修正解釋令，將票券金融公司納入國際金融業務條例所稱之「其他金融機構」範圍。
----------------------------------------------------------------------

[72] 快訊／紙本三倍券領券破500萬人次！有折痕及髒污處理原則曝光

▲振興三倍券今（16）日為開放領取實體券第二天。（圖／記者張一中攝）
記者林淑慧／台北報導
振興三倍券今（16）日為開放領取實體券第二天，依據經濟部統計，截至今天下午10時為止，全台已有527萬6937人完成領券；官員也提醒，依振興三倍券兌付要點，振興三倍券餘留在3/4以上，只要在號碼、文字及花紋可辨識的狀況下，三倍

▲亞獅康經營團隊。（圖／翻攝櫃買中心）
記者姚惠茹／台北報導
亞獅康-KY（6497）日前公告第2季每股虧損 0.62 元，上半年每股虧損1.11元，雖然虧損較去年縮減，但每股淨值-1.36元，根據規定必須要下市，因此櫃買中心今（16）日公告自8月25日起終止櫃檯買賣。
根據公告內容，董事會確認亞獅康-KY經證券主管機關終止櫃檯買賣，為維護及協助股東實現價值，董事會授權本管理階層制訂股份全面轉換計畫，透過該轉換計畫賦予股東有權依序於未來6~12個月內將其所持股份轉換為在那斯達克掛牌的美國存託憑證，該全面轉換計畫將在董事會最終決議後公布相關細節。
營運方面，亞獅康旗下新藥，包括異位性皮膚炎藥物ASLAN004及治療急性骨髓性白血病 (AML) 的口服抑制劑ASLAN003，執行長傅勇表示，隨著新加坡逐漸解除因應新冠肺炎(COVID-19)的禁令，亞獅康預計將在8月初恢復ASLAN004多劑量遞增藥物試驗(MAD)的招募。
傅勇指出，因新冠肺炎疫情持續影響造成受試者延遲招募，亞獅康預計在第4季分享更多的試驗數據，並持續積極準備在澳洲和美國增設臨床試驗地點， 以加速受試者招募，有利於2021年展開異位性皮膚炎全球性第2b期臨床試驗。
►過期票券別浪費！簡單3步驟　教你換1.5倍東森幣
----------------------------------------------------------------------

[82] 權證避險證交稅降至千分之1　擬實施5年

▲鼓勵券商造市，財政部修正權證避險證交稅擬從千分之三降至千分之一。（圖／來源自Pixta）
記者吳靜君／台北報導
財政部近日預告《證券交易稅條例》修正案，為了鼓勵券商積極造市與避險，財政部修正券商權證避險證交稅從千分之三降至千分之一，實施日期擬為五年，如果明年可以順利上路的話，實施期間可以到2025年。
根據金管會稅式支出報告，2014年到2017年的權證交易年平均交易金額為7519億元，若調降證交稅到千分之一，交易金額可望擴增為1兆3384億元，成長1.78倍。
財政部表示，如果按照金管會的報告，降稅之後可透過造市效果活絡權證交易市場，證交稅損轉為稅收增加，也可挹注我國的證交稅、營所稅、綜所稅等合計8億元。
至於實施五年的原因，財政部解釋，依據納保法的規範稅法或其他法律為特定政策所規定的租稅優惠，

文／洪寶山
今年的中國內憂外患，外部有貿易戰，內部有疫情與水災，經濟衰退迫在眉睫，五月習近平在全國政協會議經濟小組提出「內循環經濟」一詞，六月劉鶴在陸家嘴論壇致詞時再一次提出「內循環經濟」，等同於把半導體的「國產替代」概念，進一步延伸到所有產業，七月中國國家發改委等十三部委發布《關於支援新業態新模式健康發展，啟動消費市場帶動擴大就業意見》。
具體來看，在積極探索線上服務新模式，啟動消費新市場方面提出四項，包括大力發展融合化線上教育、積極發展互聯網醫療、鼓勵發展便捷化線上辦公、不斷提升數位化治理水準。
在加快推進產業數位化轉型，壯大實體經濟新動能方面則提出四項，包括培育產業平台化發展生態、加快傳統企業數位化轉型步伐、打造跨越物理邊界的「虛擬」產業園和產業聚落、發展基於新技術的「無人經濟」。
簡單說，中國正在培育新的就業形態，帶動多元投資，十五種新業態新模式形成強大內需市場，以國內循環為主、國際國內互促的雙循環發展的新格局邁進。
加強虛擬經濟創造就業機會
在全球疫情持續擴散，入出境隔離十四天的政策無法解除的情況下，各國出口都受到影響，因此如何提振內需來彌補外銷，成為各國政府的施政方向，中國提出的版本就是延續新經濟的七大領域的進一步升級，包括5G、AI、大數據、工業互聯網、特高壓、城際軌道交通與新能源汽車充電樁，以領先全球的數字虛擬經濟創造就業機會，來彌補實體經濟受疫情而導致的失業人口。中國的作法能提供其他國家什麼樣的借鏡呢？
中國是內陸型國家，不同於台灣是海島型國家，中國地大物博，單靠內需市場的國產替代或許可以延緩經濟衰退一陣子，但台灣天然資源短缺，能否自絕於世界貿易體系之外，學中國搞起內循環經濟支撐出口，這就不一定了，這點可以參考一下同樣是海島型國家的英國的現況，對比台灣或許可以參考借鏡。
英國在脫歐前，沒有貿易關稅的問題，英國央行多次警告，如果無協議脫歐，則英國GDP衰退2%，有可能陷入負值，因為無協議退出歐盟後，將從根本上改變英國與歐盟之間的進出口規則，導致英國的產業獲利困難。雖然台灣目前沒有貿易關稅的問題，但是因疫情而實施的入出境十四天的隔離觀察政策，其殺傷力也不小，兩者都跟疫情前的自由貿易狀態大不相同，可以知道，合作還是好過單打獨鬥。
英國的苦日子恐怕才剛開始
如果英國單靠金融與服務業來支撐經濟，從上述的數據展望可知，民眾要過苦日子，而且可能還不止一至二年。同

In [7]:
browser.quit()